In [26]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [27]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [28]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period','koi_period_err1','koi_period_err2', 'koi_time0bk','koi_time0bk_err1','koi_srad','koi_srad_err1','koi_srad_err2' ,'koi_kepmag'  ]]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [29]:
# In this case, they want you to set up koi_disposition as the y feature
# and the rest of the dataframe as the X feature

from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

#removed , stratify=y

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [30]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1       False_Positive
2       False_Positive
3            CONFIRMED
4            CONFIRMED
             ...      
6986    False_Positive
6987    False_Positive
6988         CANDIDATE
6989    False_Positive
6990    False_Positive
Name: koi_disposition, Length: 6991, dtype: object

In [31]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [32]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(X_test_scaled)

[[1.         0.         0.         ... 0.71935671 0.78182436 0.7335317 ]
 [0.         0.         0.         ... 0.81490723 0.65788966 0.67575833]
 [0.         0.         1.         ... 0.52590633 0.93147304 0.74427639]
 ...
 [0.         0.         0.         ... 0.69767242 0.2535126  0.58550293]
 [0.         0.         0.         ... 0.67275473 0.27116045 0.63294487]
 [0.         0.         0.         ... 0.56882006 0.34870286 0.62269609]]


In [33]:
# One-hot encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_enc=LabelEncoder()

label_enc.fit(y_train)

encoded_ytrain =label_enc.transform(y_train)

encoded_ytest =label_enc.transform(y_test)


y_train_categorical = to_categorical(encoded_ytrain)
y_test_categorical = to_categorical(encoded_ytest)
y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [34]:
y_train_categorical.shape

(5243, 3)

In [35]:
# start with a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))


In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Use categorical crossentropy for categorical data and mean squared error for regression
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:

from keras.callbacks import EarlyStopping

# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 1s - loss: 0.5415 - accuracy: 0.7316
Epoch 2/60
164/164 - 0s - loss: 0.3604 - accuracy: 0.8201
Epoch 3/60
164/164 - 0s - loss: 0.3481 - accuracy: 0.8236
Epoch 4/60
164/164 - 0s - loss: 0.3357 - accuracy: 0.8301
Epoch 5/60
164/164 - 0s - loss: 0.3385 - accuracy: 0.8257
Epoch 6/60
164/164 - 0s - loss: 0.3297 - accuracy: 0.8396
Epoch 7/60
164/164 - 0s - loss: 0.3239 - accuracy: 0.8438
Epoch 8/60
164/164 - 0s - loss: 0.3140 - accuracy: 0.8514
Epoch 9/60
164/164 - 0s - loss: 0.3110 - accuracy: 0.8514
Epoch 10/60
164/164 - 0s - loss: 0.3109 - accuracy: 0.8545
Epoch 11/60
164/164 - 0s - loss: 0.3018 - accuracy: 0.8644
Epoch 12/60
164/164 - 0s - loss: 0.3009 - accuracy: 0.8579
Epoch 13/60
164/164 - 0s - loss: 0.3024 - accuracy: 0.8608
Epoch 14/60
164/164 - 0s - loss: 0.2940 - accuracy: 0.8646
Epoch 15/60
164/164 - 0s - loss: 0.2955 - accuracy: 0.8640
Epoch 16/60
164/164 - 0s - loss: 0.2900 - accuracy: 0.8730
Epoch 17/60
164/164 - 0s - loss: 0.2879 - accuracy: 0.8718
Epoch 

In [39]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2829 - accuracy: 0.8799
Normal Neural Network - Loss: 0.2829405963420868, Accuracy: 0.8798627257347107


In [40]:
import numpy as np
# predictions = np.argmax(model.predict(X_test_scaled), axis=-1)

# predictions = (model.predict(X_test_scaled) > 0.5).astype("int32")

In [41]:
#Try to fix ---------------------------------------------AttributeError: 'Sequential' object has no attribute 'predict_classes'
# predict_x = model.predict(X_test_scaled[:5])
# classes_x=np.argmax(predict_x,axis=1)

encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_enc.inverse_transform(encoded_predictions)


print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test_categorical[:5])}")

AttributeError: 'Sequential' object has no attribute 'predict_classes'

# Train the Model



In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# # Create the GridSearchCV model
# import numpy as np
# import warnings
# warnings.filterwarnings('ignore')
# # parameter grid
# parameters = {
#     'penalty' : ['l1','l2'], 
#     'C'       : np.logspace(-3,3,7),
#     'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
# }

In [42]:
# Train the model with GridSearch
from sklearn.svm import SVC 
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf', 'poly'),'gamma': [0.0001, 0.001, 0.01] }
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_train_scaled, y_train) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}


In [43]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.8714435412861394


In [ ]:
# # Create the SVC Model for model 2
# from sklearn.svm import SVC 
# model2 = SVC(kernel='rbf')
# from sklearn.model_selection import GridSearchCV
# param_grid = {'C': [1,10], 'kernel': ('linear', 'rbf', 'poly'),'gamma': [0.0001, 0.001, 0.01] }
# grid = GridSearchCV(model2, param_grid, verbose=3)


# model2.fit(X_train_scaled,y_train)
# # model.fit(X,y)

# Save the Model

In [44]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Rob_model1.sav'
# joblib.dump(model, filename)

TypeError: can't pickle weakref objects